In [1]:
# Hack to allow imports from directory above. Once the package is fully 
# functional, remove this and instead just install and import the package
import sys
sys.path.append("..")

import pandas as pd

from portfolio_manager.portfolio import InvestmentPortfolio

In [2]:
portfolio = InvestmentPortfolio()

In [3]:
portfolio.deposit(10)
portfolio.deposit(20)

In [4]:
portfolio.portfolio_data

[{'date': datetime.datetime(2021, 1, 2, 0, 44, 31, 694288),
  'total_deposited': 10,
  'current_portfolio_value': 10},
 {'date': datetime.datetime(2021, 1, 2, 0, 44, 31, 694358),
  'total_deposited': 30,
  'current_portfolio_value': 30}]

In [5]:
portfolio.update_portfolio_value(75)

In [6]:
portfolio.withdraw(12.50)

In [7]:
df = pd.DataFrame(portfolio.portfolio_data, columns=['date', 'total_deposited', 'current_portfolio_value'])
df

,date,total_deposited,current_portfolio_value
0,2021-01-02 00:44:31.694288,10.0,10.0
1,2021-01-02 00:44:31.694358,30.0,30.0
2,2021-01-02 00:44:33.118570,30.0,75.0
3,2021-01-02 00:44:34.154358,17.5,62.5
